In [ ]:
# Install requirements

#demucs : https://github.com/facebookresearch/demucs
!pip install demucs

#TTS : https://github.com/coqui-ai/TTS
!pip install TTS

#srt : https://github.com/cdown/srt
!pip install srt

#Others
!pip install ipython
!pip install tqdm

In [3]:
# Fetch GPT translator repo
command = f'git clone https://github.com/machinewrapped/gpt-subtrans.git ../gpt-subtrans'
print(command)
os.system(command)

git clone https://github.com/machinewrapped/gpt-subtrans.git ../gpt-subtrans


Cloning into '../gpt-subtrans'...


0

In [1]:
import torch
import  whisper
from tqdm import trange
from moviepy.editor import *
import demucs.separate
from TTS.api import TTS
from datetime import timedelta
import os
import srt
import gc
# Get device
import IPython.display as ipd
device = "cuda" if torch.cuda.is_available() else "cpu"
gpt_subtrans_path="../gpt-subtrans/gpt-subtrans.py"
# List available 🐸TTS models
print(TTS().list_models())
def clear_memory():
    gc.collect()
    torch.cuda.empty_cache()
    
def merge_clips(translated_clips,background_track,output_audio_track):
    translated_audio = CompositeAudioClip(translated_clips)
    translated_audio.clips.append(AudioFileClip(background_track))
    translated_audio.fps=16000
    translated_audio.write_audiofile(output_audio_track)

In [4]:
# The filename and extension of the original video
# Try a short video for the first job (less than 2 min), some of the following steps will take a lot of time.
filename = 'hell_s_kitchen'
file_extension = 'mp4'

# source/target code list : https://github.com/openai/whisper/blob/ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab/whisper/tokenizer.py#L10
source_language_code = 'en'
# source_language_code = 'zh'
# source_language_code = 'es'

# Target: 16 language supported
# code used in tts : run `tts.languages` after the tts initialized
# code used in ffmpeg :https://trac.ffmpeg.org/wiki/Map#Specificlanguage

# Target Chinese
# target_language = 'Simplified Chinese'
# target_language_code = 'zh'
# target_language_tts_code = 'zh-cn'
# target_track_in_ffmpeg = 'zho'



# Target Spanish
target_language = 'Spanish'
target_language_code = 'es'
target_language_tts_code = 'es'
target_track_in_ffmpeg = 'spa'

# target_language = 'English'
# target_language_code = 'en'
# target_language_tts_code = 'en'
# target_track_in_ffmpeg = 'eng'

temp = "../temp/"
input = "../input/"

# inputs
full_filename = f'{filename}.{file_extension}'
input_path = f'../input/{full_filename}'

# temporary files
input_video_path = f'{temp}{filename}_video.mp4'
input_audio_path = f'{temp}{filename}_audio.mp3'
audio_cuts_path = f"{temp}cuts/"
demucs_output_vocal=f"{temp}mdx_extra/{filename}_audio/vocals.mp3"
demucs_output_background=f"{temp}mdx_extra/{filename}_audio/no_vocals.mp3"

# outputs
output = f"../output/{filename}"
output_original_srt_path = f'{output}/subtitle_{source_language_code}.srt'
output_translated_by_gpt_srt_path = f'{output}/subtitle_{target_language_code}_gpt.srt'
output_translated_by_s2s_srt_path = f'{output}/subtitle_{target_language_code}_s2s.srt'
output_audio_s2s= f'{output}/audio_{target_language_code}_s2s.mp3'
output_audio_tts= f'{output}/audio_{target_language_code}_tts.mp3'
output_final_video= f'{output}/final_video.mp4'
output_final_tts_video= f'{output}/{filename}_tts_{target_language_code}.mp4'
output_final_s2s_video= f'{output}/{filename}_s2s_{target_language_code}.mp4'



if not os.path.isdir(output):
    os.makedirs(output)

In [ ]:
# split into video and audio track.
command = f'ffmpeg -i "{input_path}" -y -vcodec copy  -c:a copy -an "{input_video_path}" -vn "{input_audio_path}"'
print(command)
os.system(command)

In [ ]:
# Split audio track into vocal and no_vocal(background) audio
demucs.separate.main(["--mp3", "--two-stems", "vocals", "-n", "mdx_extra","-o" ,temp, input_audio_path])

In [6]:
# Detect subtitles from vocal audio.
model=None
clear_memory()
model = whisper.load_model("medium")

In [ ]:
%%time
# use demucs_output_vocal or input_audio_path?
# You should check the output since it transcribes wrong results!!!
# no_speech_threshold=0.3,logprob_threshold=-0.5,
clear_memory()
result = model.transcribe(demucs_output_vocal,verbose=True,condition_on_previous_text=False,no_speech_threshold=0.5,word_timestamps=True,language=source_language_code)

In [ ]:
# Save srt file.
segments=result['segments']
subtitles = []
audio_vacal = AudioFileClip(demucs_output_vocal)
duration = audio_vacal.duration
audio_vacal.close()
count = 0
for i in trange(len(segments)): 
    
    segment = segments[i]
    start = segment['start']
    end = segment['end']
    if start>duration:
        break
    if segment['no_speech_prob']>0.8:
        continue
    end = min(end,duration)

    if start==end or len(segment['text'])==0:
        # content = segment['text']
        # print(segment['id'])
        # print(f'start:{start} end:{start} content:{content}')
        continue
    count=count+1
    subtitle = srt.Subtitle(index=count,start=timedelta(seconds=segment['start']),end=timedelta(seconds=segment['end']),content=segment['text'])
    subtitles.append(subtitle)
subtitle_original = srt.compose(subtitles)
srt_file = open(output_original_srt_path,"w")
srt_file.write(subtitle_original)
srt_file.close()

In [ ]:
%%time
# !!!!!!!! Fill the API key !!!!!!!!!!
# translate subtitle with ChatGpt
# It'll take a long time(more than 30min for 900 sentences) because of the 'Rate limits' on the OpenAI free trial.
# You can change the limit if you have Credit Grants.
# https://platform.openai.com/account/limits
# more detail in https://github.com/machinewrapped/gpt-subtrans 
# About 0.2$ for a subtitle file with 900 sentences.
key = 'Put your openai key here(https://platform.openai.com/api-keys)'
gpt_model = 'gpt-3.5-turbo-16k'
os.environ["GPT_MODEL"] = gpt_model
limit = 3 # If you have a higher rate limit, use a larger number to speed up this process
if os.path.isfile(output_translated_by_gpt_srt_path):
    print("srt has been translated")
else:
    command = f'python "{gpt_subtrans_path}" "{output_original_srt_path}" -k {key}  --target_language "{target_language}" -o "{output_translated_by_gpt_srt_path}" -r {limit}'
    print(command)
    os.system(command)

In [ ]:
# Init TTS
if 'tts'  not in vars() or tts is None:
    tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)
clear_memory()

In [ ]:
%%time

if os.path.isdir(audio_cuts_path):
    shutil.rmtree(audio_cuts_path)
if not os.path.isdir(audio_cuts_path):
    os.makedirs(audio_cuts_path)
    
file = open(output_original_srt_path,'r')
subtitles_original = srt.parse(file.read())
subtitles_original = list(subtitles_original)
file.close()

# read translated_srt
file = open(output_translated_by_gpt_srt_path,'r')
subtitles_zh = srt.parse(file.read())
subtitles_zh = list(subtitles_zh)
file.close()

tts_speed = None
s2s_clips = []
tts_clips = []
subtitles_s2s=[]
previous_end = 0
audio_vacal = AudioFileClip(demucs_output_vocal)
duration=audio_vacal.duration
audio_vacal.close()
for i in trange(len(subtitles_zh)): 
    segment = subtitles_zh[i]
    segment_original=subtitles_original[i]
    if i+1<len(subtitles_zh):
        next_segment=subtitles_zh[i+1]
        next_start =  next_segment.start.total_seconds()
    else:
        next_start = duration
    speaker_wav = audio_cuts_path+f"{i}.wav"
    start =  segment.start.total_seconds()
    
    end = segment.end.total_seconds()
    print(f'======\nstart:{start} end:{end} \n{segment_original.content} \n{segment.content}')
    audio_vacal = AudioFileClip(demucs_output_vocal)
    audio_clip = audio_vacal.subclip(start,end)
    audio_clip.write_audiofile(speaker_wav,verbose=False,logger=None)
    audio_clip.close()
    audio_vacal.close()
    
    content = subtitles_zh[i].content
    out_wav_tts = audio_cuts_path+f"{i}_zh_tts.wav"
    out_wav_tts_fixed = audio_cuts_path+f"{i}_zh_tts_fixed.wav"

    try:
        expect_duration = end-start
        count = 0
        last_duraion = 1000000
        # Because sometimes the TTS creates audio clips that are quite long, it may be a bug of TTS
        # Try 3 times if duration < expect_duration * 1.8, and pick the shortest one.
        while count<3:
            out_wav_tts_temp = audio_cuts_path+f"{i}_zh_tts_temp.wav"
            tts.tts_to_file(text=content, speaker_wav=speaker_wav,speed=tts_speed, language=target_language_tts_code, file_path=out_wav_tts_temp)
            tts_audio_clip = AudioFileClip(out_wav_tts_temp)
            duration = tts_audio_clip.duration
            tts_audio_clip.close()
            # Pick the shortest one.
            if duration < last_duraion:
                last_duraion = duration
                shutil.copyfile(out_wav_tts_temp, out_wav_tts)
            os.remove(out_wav_tts_temp)
            if duration < expect_duration * 1.8:
                print(f'accept expect_duration:{expect_duration} duration:{duration} , time:{count}')
                break
            else:
                count = count+1
                print(f'retry expect_duration:{expect_duration} duration:{duration} , time:{count}')
                continue

        # Adjust the audio speed.
        factor = 1.0
        fixed_start = start
        diff = 0
        if fixed_start<previous_end:
            diff = previous_end-fixed_start
            fixed_start = previous_end
        if duration>expect_duration:
            expect_duration-=diff
            if expect_duration<=0:
                factor = 1.8
            else:
                factor = max(duration/expect_duration,factor)
                factor = min(factor,1.8)
        
        command = f'ffmpeg -i "{out_wav_tts}" -af atempo={factor} -hide_banner -loglevel error -y {out_wav_tts_fixed}'
        os.system(command)
        tts_audio_clip = AudioFileClip(out_wav_tts_fixed)
        duration = tts_audio_clip.duration
        temp = previous_end
        previous_end = fixed_start+duration
        tts_audio_clip=tts_audio_clip.set_start(fixed_start)
        tts_clips.append(tts_audio_clip)
        print(f'factor:{factor} previous_end:{temp} , fixed_start:{fixed_start} fixed_end:{previous_end}')

        # Debug: show original audio, translated audio and adjusted speed audio, will slow down this process
        # IPython.display.display(ipd.Audio(speaker_wav,autoplay=False) )
        # IPython.display.display(ipd.Audio(out_wav_tts,autoplay=False) )
        # IPython.display.display(ipd.Audio(out_wav_tts_fixed,autoplay=False) )
    except RuntimeError as e:
        print("error in tts")
        print(e)
    except Exception as error:
        print("error in tts")
        print(error)

In [ ]:
# Merge translated audio clips and background audio.
merge_clips(tts_clips,demucs_output_background,output_audio_tts)
for clip in tts_clips:
    clip.close()

In [ ]:
# Generator the final video with the original video track, translated audio track and translated subtitle.
# Find the video in 'output' folder.
command = f'ffmpeg -i "{input_video_path}" -i "{output_audio_tts}" -i "{output_translated_by_gpt_srt_path}" -c:v copy -c:a copy -c:s mov_text -map 0 -map 1 -map 2 -metadata:s:s:0 language={target_track_in_ffmpeg}  -movflags +faststart -y "{output_final_tts_video}"'
print(command)
os.system(command)